## This Notebook is used for making the Target Variable which needs specific information for each POI and that is Reviews, we will collect reviews from each POI map it on graph and further move on

In [ ]:
import googlemaps
import pandas as pd
import numpy as np
import time
from shapely.geometry import box
import osmnx as ox
import os

# 1. Initialize Google Maps client
API_KEY = os.getenv("key")
gmaps = googlemaps.Client(key=API_KEY)

# 2. Get Islamabad boundary
place = "Islamabad, Pakistan"
gdf = ox.geocode_to_gdf(place)
minx, miny, maxx, maxy = gdf.total_bounds

# 3. Define tile step (≈ 2km apart)
step = 0.02  # Degrees ≈ 2km spacing (adjustable)
points = [(y, x) for x in np.arange(minx, maxx, step)
                for y in np.arange(miny, maxy, step)]

# 4. Search all tiles
shop_type = 'university'
all_results = []
seen_place_ids = set()

for lat, lon in points:
    print(f"Searching: {lat:.4f}, {lon:.4f}")
    try:
        token = None
        for _ in range(3):  # Max 60 results per tile
            res = gmaps.places_nearby(location=(lat, lon), radius=1000, type=shop_type, page_token=token)
            for place in res['results']:
                if place['place_id'] not in seen_place_ids:
                    all_results.append(place)
                    seen_place_ids.add(place['place_id'])
            token = res.get('next_page_token')
            if not token:
                break
            time.sleep(2)  # required delay for token
    except Exception as e:
        print("Error:", e)
        continue
    time.sleep(1.5)  # avoid hitting API limits

# 5. Convert to DataFrame
data = []
for place in all_results:
    data.append({
        'place_id': place['place_id'],
        'name': place.get('name'),
        'lat': place['geometry']['location']['lat'],
        'lon': place['geometry']['location']['lng'],
        'rating': place.get('rating'),
        'user_ratings_total': place.get('user_ratings_total'),
        'types': ', '.join(place.get('types', []))
    })

df = pd.DataFrame(data)
df.drop_duplicates(subset='place_id', inplace=True)

# 6. Save to CSV
df.to_csv("C:/Users/tehma/OneDrive/Desktop/Shop Prediction Model/SHOP_PROJECT/All_Shops_Data_CSV/universities_full_islamabad.csv", index=False)

print(f"✅ Collected {len(df)} universities from Islamabad.")


In [ ]:
# import osmnx as ox
# import geopandas as gpd
# import numpy as np
# from shapely.geometry import box
# import folium

# # -------------------------------
# # 1. Get Islamabad Boundary
# # -------------------------------
# place_name = "Islamabad, Pakistan"
# boundary = ox.geocode_to_gdf(place_name)

# # -------------------------------
# # 2. Create Grid (Approx. 1 km²)
# # -------------------------------
# minx, miny, maxx, maxy = boundary.total_bounds
# step = 0.009  # ≈ 1 km in lat/lon degrees

# cells = [box(x, y, x + step, y + step)
#          for x in np.arange(minx, maxx, step)
#          for y in np.arange(miny, maxy, step)]

# grid = gpd.GeoDataFrame(geometry=cells, crs="EPSG:4326")

# # Clip grid to Islamabad boundary
# clipped_grid = gpd.overlay(grid, boundary, how="intersection")
# clipped_grid["grid_id"] = range(len(clipped_grid))

# # BElow creates interactive map

# center = boundary.geometry.centroid.iloc[0]
# m = folium.Map(location=[center.y, center.x], zoom_start=12, tiles="OpenStreetMap")

# # Add grid cells to map
# for _, row in clipped_grid.iterrows():
#     folium.GeoJson(
#         row["geometry"],
#         style_function=lambda x: {
#             'fillColor': 'none',
#             'color': 'black',
#             'weight': 1,
#             'fillOpacity': 0
#         },
#         tooltip=f"Grid ID: {row['grid_id']}"
#     ).add_to(m)

# # -------------------------------
# # 4. Save and View
# # -------------------------------
# m.save("C:/Users/tehma/OneDrive/Desktop/Shop Prediction Model/SHOP_PROJECT/All_HTML_Mappings/islamabad_grid_map.html")


In [ ]:
import pandas as pd
import folium
import geopandas as gpd
import osmnx as ox
import numpy as np
from shapely.geometry import box

# -------------------------------
# 1. Load Islamabad boundary and create grid
# -------------------------------
place_name = "Islamabad, Pakistan"
boundary = ox.geocode_to_gdf(place_name)

minx, miny, maxx, maxy = boundary.total_bounds
step = 0.009

cells = [box(x, y, x + step, y + step)
         for x in np.arange(minx, maxx, step)
         for y in np.arange(miny, maxy, step)]

grid = gpd.GeoDataFrame(geometry=cells, crs="EPSG:4326")
clipped_grid = gpd.overlay(grid, boundary, how="intersection")
clipped_grid["grid_id"] = range(len(clipped_grid))

# -------------------------------
# 2. Load university data from CSV
# -------------------------------
df_uni = pd.read_csv("C:/Users/tehma/OneDrive/Desktop/Shop Prediction Model/SHOP_PROJECT/All_Shops_Data_CSV/universities_full_islamabad.csv")

# -------------------------------
# 3. Initialize folium map
# -------------------------------
center = boundary.geometry.centroid.iloc[0]
m = folium.Map(location=[center.y, center.x], zoom_start=12, tiles="OpenStreetMap")

# -------------------------------
# 4. Add grid cells
# -------------------------------
for _, row in clipped_grid.iterrows():
    folium.GeoJson(
        row["geometry"],
        style_function=lambda x: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0
        },
        tooltip=f"Grid ID: {row['grid_id']}"
    ).add_to(m)

# -------------------------------
# 5. Add university markers with flag icon
# -------------------------------
for _, row in df_uni.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['name'],
        tooltip=row['name'],
        icon=folium.Icon(icon='flag', color='red')
    ).add_to(m)

# -------------------------------
# 6. Save the map
# -------------------------------
m.save("C:/Users/tehma/OneDrive/Desktop/Shop Prediction Model/SHOP_PROJECT/All_HTML_Mappings/islamabad_grid_with_universities.html")



In [ ]:
m